## Zebrafish_Mature miRNA Isolated Heatmap
This is the same heatmap as the Zebrafish_mature miRNA Seed Heatmap, but only miRNA that are present in the Zebrafish genome are shown. 

In [4]:
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt

import numpy as np
from numpy import loadtxt

from collections import Counter

from sklearn import preprocessing

SIZE = 7
def sequence_count(string, dictionary):
    for i in range(len(string)-(SIZE-1)):
        temp_seq = string[i:i+(SIZE)]
        dictionary[temp_seq]+= 1
    return dictionary

def print_map(filename, map_):
    df=pd.DataFrame(map_)
    df.to_csv(filename+'.csv', index= False, header = False)



The first step it to take the dre_mature.txt and return a list of seeds. They are saved into file "dre_mature_seeds.txt"

In [2]:
from numpy import loadtxt
import pandas as pd
def getSeed(filename, x1, x2):
    #import file
    filename_data = loadtxt(filename+".txt", comments=">", delimiter="\z",dtype="str")
    
    #substring from x1, x2
    filename_data = [sub[x1:x2] for sub in filename_data]

    return filename_data

def print_text(filename, arr):
    with open(filename+".txt", "w+") as file:
        for item in arr:
            file.write("%s\n" % item)
    file.close()

print_text("dre_mature_seeds", getSeed("dre_mature", 1,8))

Next we generate the seed-matches using the dr_mature_seeds. The seed matches will be present in the Zebrafish genome. 

In [2]:
from numpy import loadtxt
import pandas as pd
def reverse_compliment(filename):
    #import file
    filename_data = loadtxt(filename+".txt", comments=">", delimiter="\z",dtype="str")


    #Replace A with X
    filename_data = [sub.replace('A', 'X') for sub in filename_data]
    
    #Replace T with A
    filename_data = [sub.replace('U', 'A') for sub in filename_data]
    
    #Replace X with T
    filename_data = [sub.replace('X', 'U') for sub in filename_data]
    
    #Replace C with X
    filename_data = [sub.replace('C', 'X') for sub in filename_data]
    
    #Replace G with C
    filename_data = [sub.replace('G', 'C') for sub in filename_data]
    
    #Replace X with G
    filename_data = [sub.replace('X', 'G') for sub in filename_data]
    
    #Reverse all the strings
    filename_data_reverse = [sub[::-1] for sub in filename_data]
    
    return filename_data_reverse

def print_text(filename, arr):
    with open(filename+".txt", "w+") as file:
        for item in arr:
            file.write("%s\n" % item)
    file.close()

print_text("dre_mature_match", reverse_compliment("dre_mature_seeds"))

Using the 7seq map from the Zebrafish Seed Match project, we can create a heatmap with the abundance of seeds. In this map, all values of zero are black. 

In [1]:
def make_map(filename):
    filename_data = loadtxt(filename+".txt", comments=">", delimiter="\z",dtype="str")
    
   
    #create the dictionary
    df = pd.read_csv("AGTC-7seq.csv", header = None)
    data_arr = np.array(df).flatten()

    data_dict = {data_arr[i]: 0 for i in range(0, len(data_arr))}

    #Update the dictionary:
    for i in filename_data:
        data_dict= sequence_count(i,data_dict)

    #Convert dictionary back into 1D array
    data_arr1 = np.array(list(data_dict.values()))

    #Convert 1D array back into a 2D array
    data_arr2 = np.reshape(data_arr1, (2**SIZE,2**SIZE))

    #save the map as a csv
    print_map(filename+"_map", data_arr2)

    #map 2D array
    df = pd.read_csv(filename+"_map.csv", header = None)
    
    #set all 0s to black
    df= np.ma.masked_where(df<0.5, df)
    cmap_reds = mpl.cm.get_cmap("Reds")
    cmap_reds.set_bad(color='black')

     #standardized the data with mean and std, then putting it through a sigmoid function 
    def sigmoid(x):
        e = np.exp(1)
        y = 1/(1+e**(-x))
        return y

    df = (df - df.mean())/(df.std())
    
    df = sigmoid(df)

    fig = plt.figure(figsize=(25,25))
    plt.imshow(df,cmap=cmap_reds)
    plt.rc('font', size=20)
    plt.title(filename+" Map")
    plt.colorbar()
    plt.savefig(filename+" Map")
    plt.show()
    
make_map("dre_mature_match")

NameError: name 'loadtxt' is not defined